In [ ]:
from csgo.parser import DemoParser
from csgo.analytics import stats
import numpy
import csv
import pandas
import pyserialem
import os
from tabulate import tabulate

In [ ]:
def getDemoPaths(directory):
    demoPaths = []
    for filename in os.listdir(directory):
        demoPaths.append(os.path.join(directory, filename))
    return demoPaths

In [ ]:
def generateDemoParsers(paths):
    demo_parsers = []
    for x in paths:
        demoParser = DemoParser(demofile=x, demo_id = x.split('/')[-1].split('.')[0].split('_')[0], parse_rate=128)
        demo_parsers.append(demoParser)
    print("# Demo Parsers: " + str(len(demo_parsers)))
    return demo_parsers

In [ ]:
def generateData(parsers):
    killData, roundData, damageData, frameData, playerFrames, bombEvents, grenadeData, flashData = [], [], [], [], [], [], [], []
    for x in parsers:
        x.parse()
        killData.append(x._parse_kills(return_type="df"))
        roundData.append(x._parse_rounds(return_type="df"))
        damageData.append(x._parse_damages(return_type="df"))
        frameData.append(x._parse_frames(return_type="df"))
        playerFrames.append(x._parse_player_frames(return_type="df"))
        bombEvents.append(x._parse_bomb_events(return_type="df"))
        grenadeData.append(x._parse_grenades(return_type="df"))
        flashData.append(x._parse_flashes(return_type="df"))
    return (killData, roundData, damageData, frameData, playerFrames, bombEvents, grenadeData, flashData)

In [ ]:
def concat_data(aggregate_data):
    final_df = None
    for x in aggregate_data:
        final_df = pandas.concat([final_df, x], ignore_index=True)
    return final_df

In [ ]:
def divideDataByMaps(data):
    dataByMap = {}
    for x in data.MapName.unique():
        dataByMap[x] = data[data['MapName'] == x]
    return dataByMap

In [ ]:
players_of_team = [ 'Boombl4', 'Perfecto', 'b1t', 'electronic', 's1mple']

In [ ]:
demo_paths = "/mnt/d/ESL_COLOGNE"

In [ ]:
killData, roundData, damageData, frameData, playerFrames, bombEvents, grenadeData, flashData = generateData(generateDemoParsers(getDemoPaths(demo_paths)))

In [ ]:
finalKillData = concat_data(killData)
finalRoundData = concat_data(roundData)
finalDamageData = concat_data(damageData)
finalGrenadeData = concat_data(grenadeData)
finalFlashData = concat_data(flashData)

In [ ]:
killMapData = divideDataByMaps(finalKillData)
roundMapData = divideDataByMaps(finalRoundData)
damageMapData = divideDataByMaps(finalDamageData)
grenadeMapData = divideDataByMaps(finalGrenadeData)
flashMapData = divideDataByMaps(finalFlashData)

In [ ]:
adr_stat = stats.adr(finalDamageData, finalRoundData, damage_filters = {"AttackerTeam" : ["==", "Natus Vincere"]})

In [ ]:
adr_stat

In [ ]:
team_1_adr_stats = {}
for x in range(len(damageData)):
    team_1_adr_stats[roundData[x]['MatchId'].unique()[0]] = stats.adr(damageData[x], roundData[x], damage_filters = {"AttackerTeam" :
                                                                                                             ["==", "Natus Vincere"]})

## Break

In [ ]:
round_end_reasons = {}
for x in killMapData.keys():
    round_end_reasons[x] = stats.win_breakdown(roundMapData[x])

In [ ]:
round_end_reasons['de_dust2']

In [ ]:
round_end_reasons['de_mirage']

In [ ]:
round_end_reasons['de_nuke']

In [ ]:
round_end_reasons['de_ancient']

In [ ]:
round_end_reasons['de_inferno']

In [ ]:
round_end_reasons['de_overpass']

## Break

In [ ]:
util_breakdown = {}
for x in killMapData.keys():
    util_breakdown[x] = stats.util_dmg_breakdown(damageMapData[x], grenadeMapData[x])

In [ ]:
util_breakdown['de_dust2'][util_breakdown['de_dust2']['Player'].isin(players_of_team)]

In [ ]:
util_breakdown['de_mirage'][util_breakdown['de_mirage']['Player'].isin(players_of_team)]

In [ ]:
util_breakdown['de_nuke'][util_breakdown['de_nuke']['Player'].isin(players_of_team)]

In [ ]:
util_breakdown['de_ancient'][util_breakdown['de_ancient']['Player'].isin(players_of_team)]

In [ ]:
util_breakdown['de_inferno'][util_breakdown['de_inferno']['Player'].isin(players_of_team)]

In [ ]:
util_breakdown['de_overpass'][util_breakdown['de_overpass']['Player'].isin(players_of_team)]

## Break

In [ ]:
util_dmg = {}
for x in killMapData.keys():
    util_dmg[x] = stats.util_dmg(damageMapData[x], grenadeMapData[x])

In [ ]:
killMapData.keys()

In [ ]:
util_dmg['de_dust2'][util_dmg['de_dust2']['Player'].isin(players_of_team)]

In [ ]:
util_dmg['de_mirage'][util_dmg['de_mirage']['Player'].isin(players_of_team)]

In [ ]:
util_dmg['de_nuke'][util_dmg['de_nuke']['Player'].isin(players_of_team)]

In [ ]:
util_dmg['de_ancient'][util_dmg['de_ancient']['Player'].isin(players_of_team)]

In [ ]:
util_dmg['de_inferno'][util_dmg['de_inferno']['Player'].isin(players_of_team)]

In [ ]:
util_dmg['de_overpass'][util_dmg['de_overpass']['Player'].isin(players_of_team)]

## Break

In [ ]:
kill_stats = {}
for x in killMapData.keys():
    kill_stats[x] = stats.kill_stats(killMapData[x], roundMapData[x])

In [ ]:
kill_stats['de_dust2'][kill_stats['de_dust2']['Player'].isin(players_of_team)]

In [ ]:
kill_stats['de_mirage'][kill_stats['de_mirage']['Player'].isin(players_of_team)]

In [ ]:
kill_stats['de_nuke'][kill_stats['de_nuke']['Player'].isin(players_of_team)]

In [ ]:
kill_stats['de_ancient'][kill_stats['de_ancient']['Player'].isin(players_of_team)]

In [ ]:
kill_stats['de_inferno'][kill_stats['de_inferno']['Player'].isin(players_of_team)]

In [ ]:
kill_stats['de_overpass'][kill_stats['de_overpass']['Player'].isin(players_of_team)]

## Break

In [ ]:
kill_breakdown = {}
for x in killMapData.keys():
    kill_breakdown[x] = stats.kill_breakdown(killMapData[x])

In [ ]:
kill_breakdown['de_dust2'][kill_breakdown['de_dust2']['Player'].isin(players_of_team)]

In [ ]:
kill_breakdown['de_mirage'][kill_breakdown['de_mirage']['Player'].isin(players_of_team)]

In [ ]:
kill_breakdown['de_nuke'][kill_breakdown['de_nuke']['Player'].isin(players_of_team)]

In [ ]:
kill_breakdown['de_ancient'][kill_breakdown['de_ancient']['Player'].isin(players_of_team)]

In [ ]:
kill_breakdown['de_inferno'][kill_breakdown['de_inferno']['Player'].isin(players_of_team)]

In [ ]:
kill_breakdown['de_overpass'][kill_breakdown['de_overpass']['Player'].isin(players_of_team)]

## Break

In [ ]:
flash_breakdown = {}
for x in killMapData.keys():
    flash_breakdown[x] = stats.flash_stats(flashMapData[x], grenadeMapData[x])

In [ ]:
killMapData.keys()

In [ ]:
flash_breakdown['de_dust2'][flash_breakdown['de_dust2']['Player'].isin(players_of_team)].drop(columns="TF")

In [ ]:
flash_breakdown['de_mirage'][flash_breakdown['de_mirage']['Player'].isin(players_of_team)].drop(columns="TF")

In [ ]:
flash_breakdown['de_nuke'][flash_breakdown['de_nuke']['Player'].isin(players_of_team)].drop(columns="TF")

In [ ]:
flash_breakdown['de_inferno'][flash_breakdown['de_inferno']['Player'].isin(players_of_team)].drop(columns="TF")

In [ ]:
flash_breakdown['de_ancient'][flash_breakdown['de_ancient']['Player'].isin(players_of_team)].drop(columns="TF")

In [ ]:
flash_breakdown['de_overpass'][flash_breakdown['de_overpass']['Player'].isin(players_of_team)].drop(columns="TF")